In [3]:
import pandas as pd, altair as alt, country_converter as coco, json

In [4]:
# import data from eurostat and unpackage from gzip compression
df = pd.read_csv('raw_data/ttr00003_page_linear.csv.gz', compression='gzip')

In [5]:
# Convert country codes to country names
df['country'] = coco.convert(names=df['geo'], to='name_short', not_found=None)

EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2
EL not found in ISO2


In [6]:
# Drop `EL`
df = df[df['country'].notna()]

df.nunique()

In [7]:
# Drop unnecessary columns
df.drop(columns=['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'tra_infr', 'n_tracks', 'OBS_FLAG'], inplace=True)

In [8]:
# group by country and rank countries by highest average `OBS_VALUE`
df_subset = df.groupby('country').mean().reset_index().sort_values('OBS_VALUE', ascending=False)
df_subset.head(10)

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_28906/3521809868.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_subset = df.groupby('country').mean().reset_index().sort_values('OBS_VALUE', ascending=False)


,country,TIME_PERIOD,OBS_VALUE
13,Germany,2015.500000,38447.000000
11,France,2015.500000,28801.030583
36,Ukraine,2016.000000,20578.000000
26,Poland,2015.500000,19492.916667
16,Italy,2015.500000,16759.916667
37,United Kingdom,2015.909091,16238.000000
32,Spain,2015.500000,15303.236667
33,Sweden,2015.500000,10969.166667
28,Romania,2015.500000,10771.583333
35,Türkiye,2014.500000,9984.500000


In [9]:
# keep only Germany, France, Ukraine, Poland, Italy, UK, Spain
countries = ['Germany', 'France', 'Ukraine', 'Poland', 'Italy', 'United Kingdom', 'Spain']
df = df[df['country'].isin(countries)]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 81 to 421
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   geo          80 non-null     object 
 1   TIME_PERIOD  80 non-null     int64  
 2   OBS_VALUE    80 non-null     float64
 3   country      80 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.1+ KB


In [11]:
df = df.drop(columns=['geo'])

df.columns = ['date', 'value', 'country']
# convert date to datetime for year
df['date'] = pd.to_datetime(df['date'], format='%Y')

# reorder columns to `country`, `date`, `value`
df = df[['country', 'date', 'value']]

---

Read in historical railroad length dataset from excel file.

In [12]:
df_pre2013 = pd.read_excel('raw_data/hbu_railroadlines.xls', sheet_name='sheet1', skiprows=4)

In [13]:
# keep only subset countries
df_pre2013 = df_pre2013[df_pre2013['country'].isin(countries)].reset_index(drop=True)

# drop column
df_pre2013 = df_pre2013.drop(columns=['objectid'])

# rename columns
df_pre2013.columns = ['country', 'date', 'value']

# convert date to datetime for year
df_pre2013['date'] = pd.to_datetime(df_pre2013['date'], format='%Y')

In [14]:
df_pre2010 = df_pre2013[df_pre2013['date'] < '2010-01-01']

In [15]:
print(df_pre2013[df_pre2013['country'] == 'United Kingdom'].tail(3))

print(df[df['country'] == 'United Kingdom'].head(3))

            country       date    value
170  United Kingdom 1995-01-01  16914.0
176  United Kingdom 2000-01-01  16878.0
182  United Kingdom 2005-01-01  16959.0
            country       date    value
411  United Kingdom 2010-01-01  15884.0
412  United Kingdom 2012-01-01  16195.0
413  United Kingdom 2013-01-01  16195.0


After 1945, Germany data is split between West Germany and East Germany. 

Let's combine them together.

In [16]:
df_deu = pd.read_excel('raw_data/hbu_railroadlines.xls', sheet_name='sheet1', skiprows=4)
# Filter to only West & East Germany
deu = df_deu[(df_deu['country'] == 'West Germany') | (df_deu['country'] == 'East Germany')]

In [17]:
deu['Country'] = 'Germany'

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_28906/2712265605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deu['Country'] = 'Germany'


In [18]:
# group by year and sum
deu = deu.groupby(['Country', 'year']).sum().reset_index()
deu.drop(columns=['objectid'], inplace=True)

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_28906/3901018654.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  deu = deu.groupby(['Country', 'year']).sum().reset_index()


In [19]:
deu.columns = ['country', 'date', 'value']
# convert date to datetime for year
deu['date'] = pd.to_datetime(deu['date'], format='%Y')

---

Combine

In [20]:
# combine dataframes
df_main = pd.concat([df_pre2010, df], ignore_index=True)

# add in East + West Germany 
df_main = pd.concat([df_main, deu], ignore_index=True)

In [21]:
# add China
df_chn = pd.read_csv('raw_data/china.csv')
df_chn['country'] = 'China'
# reorder columns to `country`, `date`, `value`
df_chn = df_chn[['country', 'date', 'value']]
# convert date to datetime for year
df_chn['date'] = pd.to_datetime(df_chn['date'], format='%Y')

# combine dataframes
df_main = pd.concat([df_main, df_chn], ignore_index=True)

# convert datetime to string for yyyy-mm-dd
df_main['date'] = df_main['date'].dt.strftime('%Y-%m-%d')

In [22]:
df_main

,country,date,value
0,United Kingdom,1825-01-01,43.0
1,France,1830-01-01,31.0
2,United Kingdom,1830-01-01,157.0
3,France,1835-01-01,141.0
4,Germany,1835-01-01,6.0
...,...,...,...
315,China,2018-01-01,131651.0
316,China,2019-01-01,139926.0
317,China,2020-01-01,146330.0
318,China,2021-01-01,150739.0


In [23]:
# explore Ukraine
df_main[df_main['country'] == 'Ukraine']

# has -999 value, this is likely in place of a missing value
# let's remove any rows with this value
df_main = df_main[df_main['value'] != -999]

---

Build chart

Template Design

In [24]:
# convert from vega-lite spec to altair spec
.configure_axisX(
    domainColor='#b4c8d8',
    domainOpacity=0.5,
    grid=False,
    labelAngle=0,
    labelColor='#b4c8d8',
    labelOpacity=0.7,
    tickColor='#b4c8d8',
    tickCount=10,
    tickOpacity=0.5,
    title='',
    titleAlign='center',
    titleAnchor='middle',
    titleColor='#b4c8d8',
    titleFontSize=12,
    titleOpacity=0.8,
    titleX=207,
    titleY=-15
).configure_axisY(
    domainColor='#b4c8d8',
    domainOpacity=0.5,
    gridColor='#b4c8d8',
    gridDash=[1, 5],
    gridOpacity=0.5,
    labelColor='#b4c8d8',
    labelOpacity=0.7,
    labelPadding=5,
    tickColor='#b4c8d8',
    tickCount=8,
    tickOpacity=0.5,
    ticks=False,
    title='Ages 16 and over, % | Source: FRED through ECO API',
    titleAlign='left',
    titleAngle=0,
    titleBaseline='bottom',
    titleColor='#b4c8d8',
    titleFontSize=12,
    titleOpacity=0.8,
    titleX=0,
    titleY=-7
)

SyntaxError: invalid syntax (4126045055.py, line 2)

In [25]:
# remove Ukraine
df_main = df_main[df_main['country'] != 'Ukraine']
# remove post 2021
df_main = df_main[df_main['date'] <= '2021-01-01']
# sort by date then by country
df_main = df_main.sort_values(['date', 'country'], ascending=[True, True]).reset_index(drop=True)

# remove China
df_main = df_main[df_main['country'] != 'China']
df_recent = df_main.groupby('country').tail(1).reset_index(drop=True)
df_recent

,country,date,value
0,France,2021-01-01,27057.00
1,Germany,2021-01-01,39799.00
2,Italy,2021-01-01,16832.00
3,Poland,2021-01-01,19287.00
4,Spain,2021-01-01,16280.46
5,United Kingdom,2021-01-01,16316.00


In [38]:
# add a transform filter for values after 1970

chart = alt.Chart(df_main).mark_line(interpolate='monotone', point='transparent').encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor='#122b39',
            tickColor='#122b39',
            domainColor='#122b39',
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            # add label expression to append 'km' to values
            # vega-lite equivalent is "labelExpr":"format(datum.value, '.0s')+'km'"
            # labelExpr="datum.value + 'km'",
            labelColor='#122b39',
            tickColor='#122b39',
            domainColor='#122b39',
            gridColor='#122b39',
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title='Total active railway track, km | Source: Eurostat & B.Mitchell (2007)',
            titleAlign='left',
            titleAngle=0,
            titleBaseline='bottom',
            titleColor='#122b39',
            titleFontSize=12,
            titleOpacity=0.9,
            titleX=0,
            titleY=-7
        )
    ),
    color=alt.Color('country:N', legend=alt.Legend(orient='top-left', labelFontSize=11, labelColor='#122b39', titleColor='#122b39', title=None, symbolSize=50)),
).transform_calculate(
    year='year(datum.date)'
).transform_filter(
    alt.datum.country != 'China' 
).properties(
    width=400,
    height=300,
    title= 'Railway Across Europe'
)

# .transform_filter(
#     alt.datum.year >= '1950'
# )


### Let's add Country name as text to the end of line (most recent data point)
# create a dataframe with the most recent value for each country
df_recent = df_main.groupby('country').tail(1).reset_index(drop=True)

# # add a text mark to the chart
# for i in range(len(df_recent)):
#     chart += alt.Chart(df_recent).mark_text(
#         baseline='middle',
#         dx=25,
#         dy=-4,
#         fontSize=12,
#         fontWeight=500,
#         color='#b4c8d8',
#     ).encode(
#         x=alt.X('date:T'),
#         y=alt.Y('value:Q'),
#         text='country:N',
#         color=alt.value('#b4c8d8'),
#     ).transform_filter(
#         alt.datum.country == df_recent['country'][i]
#     )

# base = alt.Chart(df_recent).encode(
    
# )
# # add a text mark to the chart
# uk = alt.Chart(df_recent[df_recent['country']=='United Kingdom']).mark_text(baseline='middle',dx=45,dy=-4,fontSize=12,fontWeight=500,color='#b4c8d8',
# ).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q'),
#     text='country:N',
# )

# # add a text mark to the chart
# esp = alt.Chart(df_recent[df_recent['country']=='Spain']).mark_text(baseline='middle',dx=20,dy=10,fontSize=12,fontWeight=500,color='#b4c8d8',
# ).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q'),
#     text='country:N',
# )

# pol = alt.Chart(df_recent[df_recent['country']=='Poland']).mark_text(baseline='middle',dx=20,dy=-4,fontSize=12,fontWeight=500,color='#b4c8d8',
# ).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q'),
#     text='country:N',
# )
# ita = alt.Chart(df_recent[df_recent['country']=='Italy']).mark_text(baseline='middle',dx=20,dy=10,fontSize=12,fontWeight=500,color='#b4c8d8',
# ).encode(
#     x=alt.X('date:T'),
#     y=alt.Y('value:Q'),
#     text='country:N',
# )

# chart = chart + uk + esp + pol + ita

chart = chart.configure(
    font='Circular Std'
).configure_title(
    baseline='top',
    anchor='start',
    dx=33,
    fontSize=14,
    subtitleFontSize=12,
    color='#122b39',
    subtitleColor='#000000'
).configure_view(
    stroke=None
)

chart.display()

alt.Chart(...)

In [39]:
# Convert chart to dictionary
chart_dict = chart.to_dict()

# Modify width and height directly (Altair will set continuousWidth and height by default)
chart_dict['width'] = 400
chart_dict['height'] = 300

# Convert the dictionary back to JSON
vega_spec = json.dumps(chart_dict, indent=2)

# Write the JSON to a file
with open('DH_RailwayLength_Light.json', 'w') as f:
    f.write(vega_spec)

In [40]:
import vl_convert as vlc
# # save the Altair chart as png in high resolution
# chart.save('DH_RailwayLength.png')

# Create PNG image data and then write to a file
png_data = vlc.vegalite_to_png(vl_spec=vega_spec, scale=1.5)
with open("DH_RailwayLength_Light.png", "wb") as f:
    f.write(png_data)

---

No title, light version

In [27]:
# add a transform filter for values after 1970

chart = alt.Chart(df_main).mark_line(interpolate='monotone', point='transparent').encode(
    x=alt.X(
        'date:T',
        axis=alt.Axis(
            labelColor='#122b39',
            tickColor='#122b39',
            domainColor='#122b39',
            domainOpacity=0.5,
            grid=False,
            labelAngle=0,
            tickCount=10,
            tickOpacity=0.5,
            title=None
        )
    ),
    y=alt.Y(
        'value:Q',
        axis=alt.Axis(
            # add label expression to append 'km' to values
            # vega-lite equivalent is "labelExpr":"format(datum.value, '.0s')+'km'"
            # labelExpr="datum.value + 'km'",
            labelColor='#122b39',
            tickColor='#122b39',
            domainColor='#122b39',
            gridColor='#122b39',
            gridDash=[1, 5],
            gridOpacity=0.5,
            labelPadding=5,
            tickCount=8,
            tickOpacity=0.5,
            ticks=False,
            title=None
        )
    ),
    color=alt.Color('country:N', legend=alt.Legend(orient='top-left', labelFontSize=11, labelColor='#122b39', titleColor='#122b39', title=None, symbolSize=50)),
).transform_calculate(
    year='year(datum.date)'
).transform_filter(
    alt.datum.country != 'China' 
).properties(
    width=400,
    height=300
)

### Let's add Country name as text to the end of line (most recent data point)
# create a dataframe with the most recent value for each country
df_recent = df_main.groupby('country').tail(1).reset_index(drop=True)

chart = chart.configure(
    font='Circular Std'
).configure_view(
    stroke=None
)

chart.display()

alt.Chart(...)

In [28]:
# Convert chart to dictionary
chart_dict = chart.to_dict()

# Modify width and height directly (Altair will set continuousWidth and height by default)
chart_dict['width'] = 400
chart_dict['height'] = 300

# Convert the dictionary back to JSON
vega_spec = json.dumps(chart_dict, indent=2)

# Write the JSON to a file
with open('RailwayLength_Light_NoTitle.json', 'w') as f:
    f.write(vega_spec)

In [29]:
import vl_convert as vlc
# # save the Altair chart as png in high resolution
# chart.save('DH_RailwayLength.png')

# Create PNG image data and then write to a file
png_data = vlc.vegalite_to_png(vl_spec=vega_spec, scale=1.5)
with open("RailwayLength_Light_NoTitle.png", "wb") as f:
    f.write(png_data)